<a href="https://colab.research.google.com/github/adrj67/Aprendizaje_NO_Supervisado/blob/miprim/Mi5_Aprendizaje_No_Supervisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">𝔸ℝ𝔾𝔼ℕ𝕋𝕀ℕ𝔸 ℙℝ𝕆𝔾ℝ𝔸𝕄𝔸 </h1>
<h1 align="center">≍≍≍≍≍≍≍≍≍≍≍≍≍≍≍≍≍≍≍≍≍ </h1>
<h1 align="center"> 𝕂𝕄𝕖𝕒𝕟𝕤 - 𝕀𝕀</h1>

---

# <h1 align="center"><b>Librerias</b></h1>

---

In [ ]:
#Manipulacion de datos, arreglos, etc
import pandas as pd
import numpy as np

#Visualizacion
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Framework
from sklearn.datasets import make_blobs #Crear datasets sinteticos
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler #Para escalar los datos
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

## <h3 align="center">Funciones</h3>
<h3 align="center">𝝽⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔𝝽</h3>

In [ ]:
#======================================================================
# Funcion para graficar los clusters con matplotlib
#======================================================================
def plot_clusters_matplotlib(x, y, c, axe, title):
  #Seteamos valores de la grafica
  axe.set_title(title)
  axe.set_xlabel(f"{x.columns[0]}")
  axe.set_ylabel(f"{x.columns[1]}")
  #Obtenemos valores para graficar
  y_uniques = pd.Series(y).unique()
  x = np.array(x)
  y = np.array(y)
  for y_unique in y_uniques:
    axe.scatter(x[y == y_unique, 0], x[y == y_unique, 1], label=y_unique)
    axe.markers = [f"${y_unique}$"]
    axe.legend()
  axe.scatter(c[:, 0], c[:, 1], marker='x', s=200, linewidths=3, color='black', label='Centroids')
  axe.legend()

#======================================================================
# Funcion para graficar los clusters con plotly
#======================================================================
def plot_clusters_plotly(x, y, c, title):
  #Grafica
  fig = px.scatter(x, x=x.columns[0],
                   y=x.columns[1],
                   color=y,
                   title=title)
  #Agremos los cluster
  unique_labels = pd.Series(y).unique()
  for label in unique_labels:
      points = x[y == label]
      fig.add_trace(px.scatter(points, x=x.columns[0], y=x.columns[1], color=y).data[0])

  #Colocamos los centroides
  centroids_trace = px.scatter(c, x=x.columns[0], y=x.columns[1], color=unique_labels)
  centroids_trace.update_traces(marker=dict(symbol='x', size=12))
  fig.add_trace(centroids_trace.data[0])

  fig.show()

#======================================================================
# Funcion para graficar la matriz de correlacion con plotly
#======================================================================
def corr_matrix(df):
  # Calcular la matriz de correlación
  correlation_matrix = df.corr().round(2)

  # Crear una visualización interactiva usando Plotly
  fig = px.imshow(correlation_matrix,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                color_continuous_scale='Viridis',
                text_auto=True)
  fig.update_layout(title='Matriz de correlacion', showlegend=False, height=1000, width=1000)
  fig.show()

#======================================================================
# Funcion para graficar el metodo del codo y el coeficiente de silueta
#======================================================================
def elbow_silhouette(data, max_clusters):
  wcss = []
  sil = []
  n = np.arange(2,max_clusters)
  for i in n:
    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)
    y = kmeans.fit_predict(data)
    wcss.append(kmeans.inertia_)
    sil.append(silhouette_score(data,y))

  #Creamos las graficas
  fig = fig = make_subplots(rows=2, cols=1, shared_xaxes=True ,
                            x_title="Cantidad de clusters",
                            subplot_titles=("Elbow Method", "Silhouette Score"))
  fig.add_trace(go.Scatter(x=n,y=wcss,
                  mode='lines',
                  name="Inercia"),
                  row=1,
                  col=1)
  fig.add_trace(go.Scatter(x=n,y=sil,
                  mode='lines',
                  name="Score"),
                  row=2,
                  col=1)
  fig.update_layout(title='Elbow and Silhouette', showlegend=False)

  fig['layout']['yaxis']['title']='Inercia'
  fig['layout']['yaxis2']['title']='Silhouette Score'

  fig.show()

#======================================================================
# Funcion para el mapa mundi
#======================================================================
def get_alpha_3(location):
    try:
        return pycountry.countries.get(name=location).alpha_3
    except:
        return None

---

# <h1 align="center"><b>Analisis</b></h1>

---

In [ ]:
from google.colab import drive #Importamos la libreria drive
drive.mount("/content/drive") #Montamos el drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Cargamos el data set
df_raw = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/db/marketing_campaign.csv", sep=";")

In [ ]:
df_raw.head(5)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-09-04,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-03-08,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-02-10,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [ ]:
#Cambiamos el tipo de dato objeto
df_raw = df_raw.astype({"Education":"category", "Marital_Status":"category", "Dt_Customer":"category"})

In [ ]:
#Vemos una descripcion rapida de los datos numericos
df_raw.describe().round(2)

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.00,2240.00,2216.00,2240.00,2240.00,2240.00,2240.00,2240.00,2240.00,2240.00,...,2240.00,2240.00,2240.00,2240.00,2240.00,2240.00,2240.00,2240.0,2240.0,2240.00
mean,5592.16,1968.81,52247.25,0.44,0.51,49.11,303.94,26.30,166.95,37.53,...,5.32,0.07,0.07,0.07,0.06,0.01,0.01,3.0,11.0,0.15
std,3246.66,11.98,25173.08,0.54,0.54,28.96,336.60,39.77,225.72,54.63,...,2.43,0.26,0.26,0.26,0.25,0.11,0.10,0.0,0.0,0.36
min,0.00,1893.00,1730.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.0,11.0,0.00
25%,2828.25,1959.00,35303.00,0.00,0.00,24.00,23.75,1.00,16.00,3.00,...,3.00,0.00,0.00,0.00,0.00,0.00,0.00,3.0,11.0,0.00
50%,5458.50,1970.00,51381.50,0.00,0.00,49.00,173.50,8.00,67.00,12.00,...,6.00,0.00,0.00,0.00,0.00,0.00,0.00,3.0,11.0,0.00
75%,8427.75,1977.00,68522.00,1.00,1.00,74.00,504.25,33.00,232.00,50.00,...,7.00,0.00,0.00,0.00,0.00,0.00,0.00,3.0,11.0,0.00
max,11191.00,1996.00,666666.00,2.00,2.00,99.00,1493.00,199.00,1725.00,259.00,...,20.00,1.00,1.00,1.00,1.00,1.00,1.00,3.0,11.0,1.00


In [ ]:
df_raw.describe(include="category")

,Education,Marital_Status,Dt_Customer
count,2240,2240,2240
unique,5,8,663
top,Graduation,Married,2012-08-31
freq,1127,864,12


In [ ]:
#Vemos si tenemos datos duplicados
df_raw[df_raw.duplicated()]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response


In [ ]:
df_raw.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

In [ ]:
#Veamos la proporcion de valores nulos
df_raw.isnull().melt().pipe(
    lambda df: (px.histogram(df, x="variable", color="value", text_auto=".2f", histfunc="sum" ,barnorm="percent", title="Proporcion de nulos"))
)

In [ ]:
#Veamos de donde provienen los nulos
df_raw.isnull().transpose().pipe(lambda df: px.imshow(df, title="Filas con nulos"))

In [ ]:
total_data = df_raw.shape
null_data = df_raw.dropna().shape
diff_data = df_raw.shape[0] - df_raw.dropna().shape[0]
per_data = round((diff_data / total_data[0] ) * 100, 2)

##Imprimir los datos con color en la consola
print(f"\033[1m Dimension del data set crudo: \033[36m{df_raw.shape}\033[0m")
print(f"\033[1m Dimension del data set sin nulos: \033[96m{df_raw.dropna().shape}\033[0m")
print(f"\033[1m Cantidad de datos que pierdo si decido eliminar los nulos: \033[94m{df_raw.shape[0] - df_raw.dropna().shape[0]} es igual al {per_data}% \033[0m")

 Dimension del data set crudo: (2240, 29)
 Dimension del data set sin nulos: (2216, 29)
 Cantidad de datos que pierdo si decido eliminar los nulos: 24 es igual al 1.07% 


In [ ]:
#Veamos cuales son las filas que contiene esos nulos
filter_null = df_raw.isnull().any(axis=1)
df_raw[filter_null]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
10,1994,1983,Graduation,Married,NaN,1,0,2013-11-15,11,5,...,7,0,0,0,0,0,0,3,11,0
27,5255,1986,Graduation,Single,NaN,1,0,2013-02-20,19,5,...,1,0,0,0,0,0,0,3,11,0
43,7281,1959,PhD,Single,NaN,0,0,2013-11-05,80,81,...,2,0,0,0,0,0,0,3,11,0
48,7244,1951,Graduation,Single,NaN,2,1,2014-01-01,96,48,...,6,0,0,0,0,0,0,3,11,0
58,8557,1982,Graduation,Single,NaN,1,0,2013-06-17,57,11,...,6,0,0,0,0,0,0,3,11,0
71,10629,1973,2n Cycle,Married,NaN,1,0,2012-09-14,25,25,...,8,0,0,0,0,0,0,3,11,0
90,8996,1957,PhD,Married,NaN,2,1,2012-11-19,4,230,...,9,0,0,0,0,0,0,3,11,0
91,9235,1957,Graduation,Single,NaN,1,1,2014-05-27,45,7,...,7,0,0,0,0,0,0,3,11,0
92,5798,1973,Master,Together,NaN,0,0,2013-11-23,87,445,...,1,0,0,0,0,0,0,3,11,0
128,8268,1961,PhD,Married,NaN,0,1,2013-07-11,23,352,...,6,0,0,0,0,0,0,3,11,0


In [ ]:
#Eliminamos los nulos
df_data = df_raw.dropna()
df_data.isnull().any().sum()

0

In [ ]:
#Veamos un poco mas a las variables categoricas
display(pd.DataFrame(df_data.value_counts(["Education"])))
display(pd.DataFrame(df_data.value_counts(["Marital_Status"])))

,0
Education,
Graduation,1116
PhD,481
Master,365
2n Cycle,200
Basic,54


,0
Marital_Status,
Married,857
Together,573
Single,471
Divorced,232
Widow,76
Alone,3
Absurd,2
YOLO,2


In [ ]:
pd.DataFrame(df_data.groupby(["Education","Marital_Status"]).count()["ID"])

ID
Education  Marital_Status     
2n Cycle   Absurd            0
           Alone             0
           Divorced         23
           Married          80
           Single           36
           Together         56
           Widow             5
           YOLO              0
Basic      Absurd            0
           Alone             0
           Divorced          1
           Married          20
           Single           18
           Together         14
           Widow             1
           YOLO              0
Graduation Absurd            1
           Alone             1
           Divorced        119
           Married         429
           Single          246
           Together        285
           Widow            35
           YOLO              0
Master     Absurd            1
           Alone             1
           Divorced         37
           Married         138
           Single           75
           Together        102
           Widow            11
           YOLO              0
PhD        Absurd            0
           Alone             1
           Divorced         52
           Married         190
           Single           96
           Together        116
           Widow            24
           YOLO              2

In [ ]:
#pd.DataFrame(df_data.value_counts(["Education","Marital_Status"]))

In [ ]:
df_data

In [ ]:
#Creamos una columna con la edad de los clientes
df_data["Age"] = 2020 - df_data["Year_Birth"]
#Columna con el total gastado de cada cliente
df_data["Spent"] = df_data["MntWines"]+ df_data["MntFruits"]+ df_data["MntMeatProducts"]+ df_data["MntFishProducts"]+ df_data["MntSweetProducts"]+ df_data["MntGoldProds"]
#Cambiamos el estado civil, por juntado o solo
df_data["Living_With"]=df_data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})
#Agremos una columna con el total de hijos
df_data["Children"]=df_data["Kidhome"]+df_data["Teenhome"]
#Reeorganizamos  educacion a tres categorias: No graduado, graduado, y post.
df_data["Education"]=df_data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

<ipython-input-65-519179de72e5>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-65-519179de72e5>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-65-519179de72e5>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-65-519179d

In [ ]:
#Eliminemos algunas columnas
df_data.drop(
    ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID", "Kidhome", "Teenhome"],
    axis=1,
    inplace=True
)

<ipython-input-66-d5ec8de9b254>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_data.describe()

,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Spent,Children
count,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,...,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000,2216.000000
mean,52247.251354,49.012635,305.091606,26.356047,166.995939,37.637635,27.028881,43.965253,2.323556,4.085289,...,0.073556,0.074007,0.073105,0.064079,0.013538,0.009477,0.150271,51.179603,607.075361,0.947202
std,25173.076661,28.948352,337.327920,39.793917,224.283273,54.752082,41.072046,51.815414,1.923716,2.740951,...,0.261106,0.261842,0.260367,0.244950,0.115588,0.096907,0.357417,11.985554,602.900476,0.749062
min,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,5.000000,0.000000
25%,35303.000000,24.000000,24.000000,2.000000,16.000000,3.000000,1.000000,9.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43.000000,69.000000,0.000000
50%,51381.500000,49.000000,174.500000,8.000000,68.000000,12.000000,8.000000,24.500000,2.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,396.500000,1.000000
75%,68522.000000,74.000000,505.000000,33.000000,232.250000,50.000000,33.000000,56.000000,3.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,61.000000,1048.000000,1.000000
max,666666.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,262.000000,321.000000,15.000000,27.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,127.000000,2525.000000,3.000000


In [ ]:
f = go.Figure()
for i,name in enumerate(df_data.columns):
    f.add_trace(go.Box(y=df_data[f"{name}"], name=f"{name}", opacity=0.5))
f.update_layout(height=900, width=1000, title_text="Box Plots", paper_bgcolor="#e4e9f7")

In [ ]:
# @title Histogramas de cada variable
fig = go.Figure().set_subplots(5, 5, horizontal_spacing=0.1)
i = 1
j = 1
for idx, name in enumerate(df_data.describe().columns):
  fig.add_trace(go.Histogram(x=df_data[name]),row=i, col=j)
  # Update xaxis properties
  fig.update_xaxes(title_text=f"{name}", row=i, col=j)
  fig.add_vline(
    x=df_data.describe()[f"{name}"][1].round(2),
    line_color = "red",
    line_dash = "dash",
    row=i,
    col=j).add_vline(
    x=df_data[f"{name}"].median().round(2),
    line_color = "green",
    line_dash = "dash",
    row=i,
    col=j)

  if j==5:
    i+=1
    j=1
  else:
    j+=1

fig.update_layout(title='Histogramas con mediana (verde) y media (roja)', showlegend=False,height=1000, width=1100,paper_bgcolor="#e4e9f7")
fig.show()

In [ ]:
# @title Box plot para cada variable
fig = go.Figure().set_subplots(5, 5, horizontal_spacing=0.1)
i = 1
j = 1

for idx, name in enumerate(df_data.describe().columns):
  fig.add_trace(go.Box(y=df_data[f"{name}"], name=f"{name}", opacity=0.5), row=i, col=j)
  if j==5:
    i+=1
    j=1
  else:
    j+=1

fig.update_layout(title='Histogramas con mediana (verde) y media (roja)', showlegend=False, height=1000, width=1000)
fig.show()

In [ ]:
df_data[(df_data["Age"] > 85)]

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Spent,Living_With,Children
192,Undergraduate,36640.0,99,15,6,8,7,4,25,1,...,0,0,0,0,1,0,120,65,Alone,1
239,Undergraduate,60182.0,23,8,0,5,7,0,2,1,...,0,0,0,0,0,0,127,22,Alone,1
339,Postgraduate,83532.0,36,755,144,562,104,64,224,1,...,0,1,0,0,0,0,121,1853,Partner,0


In [ ]:
df_data[(df_data["Income"] > 130000)]

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Spent,Living_With,Children
164,Postgraduate,157243.0,98,20,2,1582,1,2,1,15,...,0,0,0,0,0,0,47,1608,Partner,1
617,Postgraduate,162397.0,31,85,1,16,2,1,2,0,...,0,0,0,0,0,0,44,107,Partner,2
655,Graduate,153924.0,81,1,1,1,1,1,1,0,...,0,0,0,0,0,0,45,6,Alone,0
687,Postgraduate,160803.0,21,55,16,1622,17,3,4,15,...,0,0,0,0,0,0,38,1717,Partner,0
1300,Postgraduate,157733.0,37,39,1,9,2,0,8,0,...,0,0,0,0,0,0,49,59,Partner,1
1653,Graduate,157146.0,13,1,0,1725,2,1,1,0,...,0,0,0,0,0,0,43,1730,Partner,0
2132,Postgraduate,156924.0,85,2,1,2,1,1,1,0,...,0,0,0,0,0,0,71,8,Partner,0
2233,Graduate,666666.0,23,9,14,18,8,1,12,4,...,0,0,0,0,0,0,43,62,Partner,1


In [ ]:
#Tramos los outliers de edad e ingreso
print(df_data.shape)
df_data = df_data[(df_data["Age"] < 85)]
df_data = df_data[(df_data["Income"] < 130000)]
print(df_data.shape)

(2216, 25)
(2205, 25)


In [ ]:
corr_matrix(df_data.drop(['Living_With'], axis=1))

<ipython-input-20-92331f3c8e3f>:47: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
df_data

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,...,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Spent,Living_With,Children
0,Graduate,58138.0,58,635,88,546,172,88,88,3,...,0,0,0,0,0,1,63,1617,Alone,0
1,Graduate,46344.0,38,11,1,6,2,1,6,2,...,0,0,0,0,0,0,66,27,Alone,2
2,Graduate,71613.0,26,426,49,127,111,21,42,1,...,0,0,0,0,0,0,55,776,Partner,0
3,Graduate,26646.0,26,11,4,20,10,3,5,2,...,0,0,0,0,0,0,36,53,Partner,1
4,Postgraduate,58293.0,94,173,43,118,46,27,15,5,...,0,0,0,0,0,0,39,422,Partner,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,Graduate,61223.0,46,709,43,182,42,118,247,2,...,0,0,0,0,0,0,53,1341,Partner,1
2236,Postgraduate,64014.0,56,406,0,30,0,0,8,7,...,0,0,1,0,0,0,74,444,Partner,3
2237,Graduate,56981.0,91,908,48,217,32,12,24,1,...,1,0,0,0,0,0,39,1241,Alone,0
2238,Postgraduate,69245.0,8,428,30,214,80,30,61,2,...,0,0,0,0,0,0,64,843,Partner,1


In [ ]:
#Codifiquemos las variables categoricas
col_category = (df_data.dtypes == "category")[df_data.dtypes == "category"].index
label_encoder = LabelEncoder() #Para codificar las variables categoricas
for i in col_category:
  df_data[i] = df_data[[i]].apply(label_encoder.fit_transform)

In [ ]:
# duplocamos el dataframe para tener una copia de resguardo del dataframe
df_to_use = df_data.copy()

In [ ]:
# Borramos algunas columnas
df_to_use.drop(
    ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1','AcceptedCmp2', 'Complain', 'Response'],
    axis=1,
    inplace=True
)

In [ ]:
df_to_use

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Age,Spent,Living_With,Children
0,0,58138.0,58,635,88,546,172,88,88,3,8,10,4,7,63,1617,0,0
1,0,46344.0,38,11,1,6,2,1,6,2,1,1,2,5,66,27,0,2
2,0,71613.0,26,426,49,127,111,21,42,1,8,2,10,4,55,776,1,0
3,0,26646.0,26,11,4,20,10,3,5,2,2,0,4,6,36,53,1,1
4,1,58293.0,94,173,43,118,46,27,15,5,5,3,6,5,39,422,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,0,61223.0,46,709,43,182,42,118,247,2,9,3,4,5,53,1341,1,1
2236,1,64014.0,56,406,0,30,0,0,8,7,8,2,5,7,74,444,1,3
2237,0,56981.0,91,908,48,217,32,12,24,1,2,3,13,6,39,1241,0,0
2238,1,69245.0,8,428,30,214,80,30,61,2,6,5,10,3,64,843,1,1


In [ ]:
#Esclara el dataset
scaler = StandardScaler() #Creamos el objeto para escalar
df_scaled = scaler.fit_transform(df_to_use)

In [ ]:
df_scaled = pd.DataFrame(df_scaled, columns=df_to_use.columns)
df_scaled

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Age,Spent,Living_With,Children
0,-0.892338,0.314651,0.310830,0.974566,1.548614,1.748400,2.449154,1.480301,0.849556,0.361479,1.424772,2.628526,-0.562650,0.689203,1.017189,1.679323,-1.347625,-1.266589
1,-0.892338,-0.254877,-0.380600,-0.874776,-0.638664,-0.731678,-0.652345,-0.635399,-0.735767,-0.168834,-1.132957,-0.588043,-1.179732,-0.139645,1.273530,-0.963897,-1.347625,1.403420
2,-0.892338,0.965354,-0.795458,0.355155,0.568110,-0.175957,1.336263,-0.149031,-0.039771,-0.699147,1.424772,-0.230646,1.288596,-0.554069,0.333612,0.281242,0.742046,-1.266589
3,-0.892338,-1.206087,-0.795458,-0.874776,-0.563241,-0.667380,-0.506392,-0.586763,-0.755100,-0.168834,-0.767567,-0.945440,-0.562650,0.274779,-1.289883,-0.920675,0.742046,0.068415
4,0.571654,0.322136,1.555404,-0.394659,0.417263,-0.217292,0.150396,-0.003121,-0.561768,1.422105,0.328602,0.126750,0.054432,-0.139645,-1.033542,-0.307248,0.742046,0.068415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,-0.892338,0.463624,-0.104028,1.193879,0.417263,0.076644,0.077420,2.209853,3.923536,-0.168834,1.790162,0.126750,-0.562650,-0.139645,0.162718,1.220500,0.742046,0.068415
2201,0.571654,0.598401,0.241687,0.295881,-0.663806,-0.621452,-0.688833,-0.659718,-0.697100,2.482731,1.424772,-0.230646,-0.254109,0.689203,1.957107,-0.270675,0.742046,2.738424
2202,-0.892338,0.258780,1.451690,1.783653,0.542969,0.237389,-0.105022,-0.367897,-0.387769,-0.699147,-0.767567,0.126750,2.214218,0.274779,-1.033542,1.054260,-1.347625,-1.266589
2203,0.571654,0.851004,-1.417746,0.361082,0.090428,0.223611,0.770696,0.069834,0.327560,-0.168834,0.693992,0.841543,1.288596,-0.968493,1.102636,0.392623,0.742046,0.068415


---

## <h2 align="center"> PCA </h2>
<h4 align="center"> Analisis de componentes principales </h4>

---

In [ ]:
 #Creamos el objeto pca
pca = PCA()
pca.fit(df_scaled)

PCA()

In [ ]:
var_cum = np.cumsum(pca.explained_variance_ratio_)
pca.explained_variance_ratio_ * 100

array([3.99601679e+01, 9.92512955e+00, 6.03326543e+00, 5.80691028e+00,
       5.53975210e+00, 5.42291350e+00, 4.53590246e+00, 3.90281190e+00,
       3.34566760e+00, 3.02301013e+00, 2.45659356e+00, 2.39503364e+00,
       2.15714454e+00, 1.95865477e+00, 1.49964774e+00, 1.26541198e+00,
       7.71982900e-01, 7.79112256e-31])

In [ ]:
pca.components_

array([[-3.62537841e-02,  3.26132517e-01,  6.94310355e-03,
         2.91052985e-01,  2.62619129e-01,  3.12225953e-01,
         2.72402381e-01,  2.62332156e-01,  2.14523653e-01,
        -5.48274103e-02,  2.04994329e-01,  3.13309729e-01,
         2.78488880e-01, -2.38634639e-01,  5.28563284e-02,
         3.54838536e-01, -9.98215408e-03, -2.13470502e-01],
       [-4.31950390e-02,  5.83162408e-02,  1.13548431e-02,
         2.16949953e-01, -1.11916169e-01, -1.03772344e-01,
        -1.21649773e-01, -9.55309563e-02,  1.48315084e-01,
         6.02360413e-01,  4.31011110e-01,  3.08658044e-02,
         1.84045186e-01,  2.96092911e-01,  2.48258019e-01,
         7.18682669e-02,  4.55634385e-02,  3.71944840e-01],
       [-2.55521986e-01, -2.01681552e-01, -7.46513458e-02,
        -1.31474951e-01,  2.19420210e-01,  1.25147637e-02,
         1.73948242e-01,  1.76813411e-01,  2.83959074e-01,
         2.19955305e-01,  1.10313226e-01, -5.73865739e-02,
        -7.12224549e-02,  3.11815042e-01, -6.79859969e

In [ ]:
px.area(
    x=range(1, var_cum.shape[0] + 1),
    y=(var_cum).round(2),
    labels={"x": "Components", "y": "Explained Variance"},
    title="Varianza acumulada"
)

In [ ]:
len(df_scaled.columns)

18

In [ ]:
label = []
for i in range(len(df_scaled.columns)):
  label.append(f"PC{i+1}")
label

['PC1',
 'PC2',
 'PC3',
 'PC4',
 'PC5',
 'PC6',
 'PC7',
 'PC8',
 'PC9',
 'PC10',
 'PC11',
 'PC12',
 'PC13',
 'PC14',
 'PC15',
 'PC16',
 'PC17',
 'PC18']

In [ ]:
fig_bar = px.bar(x=label,
                 y=pca.explained_variance_ratio_*100,
                 title="Componentes principales",
                 labels={"x": "Components", "y": "Explained Variance"})
fig_bar.show()

In [ ]:
df_pca = pd.DataFrame(index =label,
                      columns=df_scaled.columns, data=pca.components_)
df_pca

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Age,Spent,Living_With,Children
PC1,-3.625378e-02,3.261325e-01,6.943104e-03,0.291053,0.262619,0.312226,0.272402,0.262332,0.214524,-5.482741e-02,2.049943e-01,3.133097e-01,2.784889e-01,-2.386346e-01,5.285633e-02,0.354839,-9.982154e-03,-2.134705e-01
PC2,-4.319504e-02,5.831624e-02,1.135484e-02,0.216950,-0.111916,-0.103772,-0.121650,-0.095531,0.148315,6.023604e-01,4.310111e-01,3.086580e-02,1.840452e-01,2.960929e-01,2.482580e-01,0.071868,4.556344e-02,3.719448e-01
PC3,-2.555220e-01,-2.016816e-01,-7.465135e-02,-0.131475,0.219420,0.012515,0.173948,0.176813,0.283959,2.199553e-01,1.103132e-01,-5.738657e-02,-7.122245e-02,3.118150e-01,-6.798600e-01,-0.002355,-2.172265e-01,-7.192152e-02
PC4,6.979729e-01,-3.553900e-02,-5.792200e-01,0.111646,-0.032896,-0.013591,-0.021545,-0.027692,-0.053579,3.006176e-02,1.098911e-01,1.603029e-03,6.848897e-02,1.100827e-01,-2.407326e-01,0.047067,2.203939e-01,-1.403896e-01
PC5,5.987160e-03,4.021501e-02,-4.005228e-01,0.041251,-0.050914,-0.009396,-0.056067,-0.082344,-0.026828,-4.493001e-02,7.069166e-03,7.162499e-03,-1.188118e-03,-2.170995e-02,2.174216e-01,0.003326,-8.789823e-01,-2.041303e-02
PC6,5.573831e-01,-4.484498e-02,7.003611e-01,0.112851,-0.078666,0.024267,-0.056578,-0.022590,-0.015505,2.591970e-02,4.975762e-02,4.222299e-02,6.286898e-03,9.485885e-02,-1.718059e-01,0.058850,-3.447878e-01,-8.669275e-02
PC7,3.429376e-01,-6.986256e-02,6.015753e-03,-0.396390,0.362851,-0.096041,0.371176,0.365683,0.164115,1.954723e-01,-1.099312e-01,-6.964853e-02,-7.986385e-02,-1.001641e-01,3.403756e-01,-0.160184,-3.640652e-02,2.581509e-01
PC8,2.637975e-02,-1.854547e-01,2.195881e-02,-0.021016,-0.084310,-0.160109,0.018474,-0.161626,0.654794,-2.863547e-01,1.982477e-01,-7.065308e-03,-1.974549e-01,2.193175e-01,3.290663e-01,-0.028379,8.263299e-02,-3.894433e-01
PC9,-8.440206e-02,-1.065084e-01,4.066913e-02,0.054362,0.194756,-0.002581,0.083521,0.326124,-0.555191,-1.657003e-01,3.513341e-01,-1.629989e-01,-7.546368e-02,4.570959e-01,2.429052e-01,0.024601,1.328637e-02,-2.560067e-01
PC10,1.006207e-02,-1.619436e-01,-4.809990e-02,0.141608,-0.035135,0.419297,0.163076,-0.129577,-0.067254,2.387475e-01,-3.821997e-01,3.372218e-01,-5.057659e-01,2.771939e-01,1.504847e-01,0.229097,4.180599e-02,6.743096e-03


In [ ]:
fig_heatmap=px.imshow(df_pca,text_auto=True, aspect='auto', title="Componentes principales")
fig_heatmap.update_coloraxes(showscale=False) # Deshabilitar la barra de colores
fig.update_layout(title='Matriz de componentes principales', showlegend=False, height=1000, width=1000)
fig_heatmap.show()

In [ ]:
df_pca_mark = pd.DataFrame(pca.transform(df_scaled), columns=label)
df_pca_mark

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18
0,4.319224,0.457944,1.333048,-0.995264,0.981242,0.303217,0.777551,0.947828,1.584624,2.268941,-0.040034,0.340065,0.423777,0.724186,-0.804038,0.327291,0.141276,-4.214174e-16
1,-2.469723,-0.151241,-0.920841,-1.461632,1.678164,-0.794854,0.539487,-0.367649,-0.276859,0.344123,0.533850,-0.015187,0.020516,0.029617,-0.122979,-0.072504,-0.089914,-1.810437e-16
2,1.985282,-0.111998,-0.182231,0.241160,-0.224715,-1.366509,-0.471146,0.540222,0.881213,-1.388040,-0.774921,-0.370483,1.169648,0.294105,0.006283,-0.309841,-0.257706,1.917961e-16
3,-2.514720,-0.903688,0.878657,0.200224,-0.586480,-1.171700,-0.712802,-0.342998,-0.061686,-0.193661,-0.302961,0.091597,0.087552,-0.007649,0.000068,-0.063390,0.323100,-6.536166e-16
4,-0.166437,0.552091,0.511292,-0.085557,-1.572261,1.254532,0.404833,-1.048224,-0.110559,-0.224178,-0.593998,-0.045948,-0.005146,0.814025,-0.254064,-0.391775,-0.494028,4.475002e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200,2.818564,1.346412,1.466132,-0.423453,-0.798671,-0.747037,0.483347,2.565423,-0.606704,-0.530434,2.175143,0.933894,-0.972123,-0.475588,0.325701,-0.813859,0.527472,6.289946e-17
2201,-1.352513,4.037889,-1.650112,-0.043963,-0.307144,-0.059859,0.931322,-0.882866,0.139118,0.293877,0.671948,-0.089732,0.230414,0.680468,0.063734,-0.245950,-0.209353,4.168958e-16
2202,1.729427,-0.610410,0.527892,-1.005624,0.521400,1.495411,-2.149369,-0.675233,0.260679,-0.584456,-1.184352,-1.084373,-0.072743,-1.774623,-0.267723,0.104894,0.098056,6.735380e-16
2203,1.924907,0.537006,-1.316592,1.149307,0.181340,-1.169861,0.490879,0.092054,-0.390066,-0.482143,-0.245920,0.086929,0.537098,0.110000,-0.205827,0.441943,0.089432,1.824223e-16


In [ ]:
# Me dice cuantos componentes debo usar para llegar al 85% o mas (en este caso)
var_cum = np.cumsum(pca.explained_variance_ratio_)*100
value = 85 # elijo a que % quiero llegar
indx = np.where(var_cum >= value)[0][0]
print(f"Numero de componenes: {indx+1} con el {var_cum[indx].round(2)}%")

Numero de componenes: 10 con el 87.5%


In [ ]:
# Borro los componentes que no voy a utilizar
drop_label = []
for i in range(8):
  drop_label.append(f"PC{i+11}")
drop_label

['PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18']

In [ ]:
# Asi queda el dataframe
df_pca_mark.drop(drop_label, axis=1,inplace=True)
df_pca_mark

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,4.319224,0.457944,1.333048,-0.995264,0.981242,0.303217,0.777551,0.947828,1.584624,2.268941
1,-2.469723,-0.151241,-0.920841,-1.461632,1.678164,-0.794854,0.539487,-0.367649,-0.276859,0.344123
2,1.985282,-0.111998,-0.182231,0.241160,-0.224715,-1.366509,-0.471146,0.540222,0.881213,-1.388040
3,-2.514720,-0.903688,0.878657,0.200224,-0.586480,-1.171700,-0.712802,-0.342998,-0.061686,-0.193661
4,-0.166437,0.552091,0.511292,-0.085557,-1.572261,1.254532,0.404833,-1.048224,-0.110559,-0.224178
...,...,...,...,...,...,...,...,...,...,...
2200,2.818564,1.346412,1.466132,-0.423453,-0.798671,-0.747037,0.483347,2.565423,-0.606704,-0.530434
2201,-1.352513,4.037889,-1.650112,-0.043963,-0.307144,-0.059859,0.931322,-0.882866,0.139118,0.293877
2202,1.729427,-0.610410,0.527892,-1.005624,0.521400,1.495411,-2.149369,-0.675233,0.260679,-0.584456
2203,1.924907,0.537006,-1.316592,1.149307,0.181340,-1.169861,0.490879,0.092054,-0.390066,-0.482143


---

# <h2 align="center"> <b>KMeans</b> </h>

---

## <h3 align="center"> Con PCA </h3>
<h3 align="center">𝝽⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔𝝽</h3>

In [ ]:
# El codo (Elbow)
elbow_silhouette(df_pca_mark, 10)

In [ ]:
#Creamos el modelo.
km_pca = KMeans(n_clusters=5, n_init=10, random_state=42)
y_predict_pca = km_pca.fit_predict(df_pca_mark) #Entrenamos el modelo y le pedimos que arme los grupos.
print(silhouette_score(df_pca_mark, y_predict_pca))

0.14652126154501047


In [ ]:
df_to_use["KMeans_pca"] = y_predict_pca
df_to_use

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Age,Spent,Living_With,Children,KMeans_pca
0,0,58138.0,58,635,88,546,172,88,88,3,8,10,4,7,63,1617,0,0,2
1,0,46344.0,38,11,1,6,2,1,6,2,1,1,2,5,66,27,0,2,1
2,0,71613.0,26,426,49,127,111,21,42,1,8,2,10,4,55,776,1,0,0
3,0,26646.0,26,11,4,20,10,3,5,2,2,0,4,6,36,53,1,1,3
4,1,58293.0,94,173,43,118,46,27,15,5,5,3,6,5,39,422,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,0,61223.0,46,709,43,182,42,118,247,2,9,3,4,5,53,1341,1,1,0
2236,1,64014.0,56,406,0,30,0,0,8,7,8,2,5,7,74,444,1,3,4
2237,0,56981.0,91,908,48,217,32,12,24,1,2,3,13,6,39,1241,0,0,0
2238,1,69245.0,8,428,30,214,80,30,61,2,6,5,10,3,64,843,1,1,0


## <h3 align="center"> Sin PCA </h>
<h3 align="center">𝝽⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔𝝽</h3>

In [ ]:
elbow_silhouette(df_scaled, 10)

In [ ]:
km = KMeans(n_clusters=3, n_init=10, random_state=42) #Creamos el modelo
y_predict = km_pca.fit_predict(df_scaled) #Entremos y le pedimos que arme los grupos.
print(silhouette_score(df_scaled, y_predict))

0.11814515247916507


In [ ]:
df_to_use["KMeans"] = y_predict
df_to_use

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Age,Spent,Living_With,Children,KMeans_pca,KMeans
0,0,58138.0,58,635,88,546,172,88,88,3,8,10,4,7,63,1617,0,0,2,3
1,0,46344.0,38,11,1,6,2,1,6,2,1,1,2,5,66,27,0,2,1,0
2,0,71613.0,26,426,49,127,111,21,42,1,8,2,10,4,55,776,1,0,0,1
3,0,26646.0,26,11,4,20,10,3,5,2,2,0,4,6,36,53,1,1,3,4
4,1,58293.0,94,173,43,118,46,27,15,5,5,3,6,5,39,422,1,1,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,0,61223.0,46,709,43,182,42,118,247,2,9,3,4,5,53,1341,1,1,0,1
2236,1,64014.0,56,406,0,30,0,0,8,7,8,2,5,7,74,444,1,3,4,2
2237,0,56981.0,91,908,48,217,32,12,24,1,2,3,13,6,39,1241,0,0,0,1
2238,1,69245.0,8,428,30,214,80,30,61,2,6,5,10,3,64,843,1,1,0,1


## <h3 align="center"> Eleccion </h>
<h3 align="center">𝝽⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔⇔𝝽</h3>

In [ ]:
np.unique(y_predict_pca) #Obtenemos los valores unicos que posee nuestras predicciones

array([0, 1, 2, 3, 4], dtype=int32)

In [ ]:
# @title Scatter plots
fig = make_subplots(rows=1, cols=3, subplot_titles=["Spent vs Income", "NumWebVisitsMonth vs Income", "NumCatalogPurchases vs Spent"])

fig.add_trace(
    go.Scatter(x=df_to_use["Spent"], y=df_to_use["Income"],
               mode="markers",
               name="Scatter 1",
               marker = dict(color=df_to_use.KMeans_pca)),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=df_to_use["NumWebVisitsMonth"], y=df_to_use["Income"],
               mode="markers",
               name="Scatter 2",
               marker = dict(color=df_to_use.KMeans_pca)),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=df_to_use["NumCatalogPurchases"], y=df_to_use["Spent"],
               mode="markers",
               name="Scatter 3",
               marker = dict(color=df_to_use.KMeans_pca)),
    row=1, col=3
)

fig.update_layout(title='Scatter plots', showlegend=False)
fig['layout']['xaxis']['title']='Spent'
fig['layout']['yaxis']['title']='Income'
fig['layout']['xaxis2']['title']='NumWebVisitsMonth'
fig['layout']['yaxis2']['title']='Income'
fig['layout']['xaxis3']['title']='NumCatalogPurchases'
fig['layout']['yaxis3']['title']='Spent'

fig.show()

# Recursos

* [Data set](https://www.kaggle.com/datasets/rodsaldanha/arketing-campaign?resource=download)
* [Notebook referencia](https://www.kaggle.com/code/karnikakapoor/customer-segmentation-clustering/notebook#DATA-PREPROCESSING)